In [ ]:
import pandas as pd
import os

In [ ]:
DATA_DIR = 'data/mimiciv/3.1'
TASK_DIR = '/data'
LABEL_COLS = ['Acute and unspecified renal failure', 'Cardiac dysrhythmias']

In [ ]:
ADM = ['subject_id', 'hadm_id', 'edregtime', 'edouttime', 'admittime', 'dischtime', 'deathtime',]
PAT = ['subject_id', 'anchor_year', 'dod']
PROC_ICD = ['subject_id', 'hadm_id', 'seq_num', 'chartdate', 'icd_code', 'icd_version']
PHARM = ['subject_id', 'hadm_id', 'poe_id', 'starttime', 'stoptime', 'medication']
EMAR = ['subject_id', 'hadm_id', 'emar_id', 'emar_seq', 'poe_id', 'charttime', 'medication']
LAB = ['labevent_id', 'subject_id', 'hadm_id', 'specimen_id', 'itemid',
       'charttime',  'value', 'valuenum', 'ref_range_lower', 'ref_range_upper', 'flag']

In [ ]:
STAY = ['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime']
CHART = ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value', 'valuenum', 'warning']
INPUT = ['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'itemid', 'amount',  
         'rate', 'patientweight', 'totalamount', 'originalamount', 'originalrate']
OUTPUT = ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value']
PROC = ['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 
            'itemid', 'value', 'patientweight', 'originalamount', 'originalrate']

In [ ]:
CHUNK_SIZE = 1_000_000

In [ ]:
def ids():
    label_df3 = pd.read_csv(os.path.join(TASK_DIR, 'in-hospital-mortality', 'test/listfile.csv'), usecols=['stay'])
    label_df4 = pd.read_csv(os.path.join(TASK_DIR, 'in-hospital-mortality', 'train/listfile.csv'), usecols=['stay'])

    label_ids = []
    label_ids += list(pd.to_numeric(label_df3['stay'].apply(lambda x: x.split('_', 1)[0])).unique())
    label_ids += list(pd.to_numeric(label_df4['stay'].apply(lambda x: x.split('_', 1)[0])).unique())
    
    label_ids = list(set(label_ids))
    return label_ids


In [ ]:
def get_csv(folder, fname, cols, label_ids):
    print(fname)
    filtered_chunks = []

    for chunk in pd.read_csv(os.path.join(DATA_DIR, folder, f'{fname}.csv'), usecols=cols, chunksize=CHUNK_SIZE):
        filtered_chunk = chunk[chunk['subject_id'].isin(label_ids)]
        filtered_chunks.append(filtered_chunk)

    df = pd.concat(filtered_chunks, ignore_index=True)
    return df

In [ ]:
all_ids = ids()
for i in range(7):
    print(f"{i}")
    label_ids = pd.read_parquet(f'data/raw_data/visits_{i}.parquet')['subject_id']

    folders = ['meta']*7
    fnames = ['emar', 'labevents',  'pharmacy', 
              'chartevents', 'inputevents', 'outputevents', 'procedureevents']
    # 'admissions', 'patients','icustays',
    cols = [EMAR, LAB, PHARM, CHART, INPUT, OUTPUT, PROC]

    for folder, fname, col in zip(folders, fnames, cols):
        df = get_csv(folder, fname, col, label_ids)
        df.to_parquet(f"data/raw_data/{fname}_{i}.parquet", index=False)

In [ ]:
import numpy as np
from scipy.stats import linregress, kurtosis, skew

In [ ]:
ROOT_DIR = 'data/'
TASK = 'in-hospital-mortality'
DATA_DIR = os.path.join(ROOT_DIR, TASK)

In [ ]:
TRAIN_FILES = os.listdir(os.path.join(DATA_DIR, 'train'))
TEST_FILES = os.listdir(os.path.join(DATA_DIR, 'test'))

TO PARQUET: EVENT, VALUE

In [ ]:
def csv_to_parquet(split, files):
    for files_i in files:
        if files_i[-4:] != '.csv' or files_i == 'listfile.csv': continue
        fname = files_i[:-4]
        df = pd.read_csv(os.path.join(DATA_DIR, f'{split}/{files_i}'))
        
        df_final = None
        for col_i in df.columns:
            if col_i == 'Hours': continue
            if not 'Hours' in df.columns: 
                print(df.columns)
                continue
            df_i = df[[col_i, 'Hours']].rename(columns={col_i:'Value'})
            df_i = df_i.dropna()
            df_i['Event'] = col_i
            if len(df_i) == 0: continue
            elif df_final is None:
                df_final = df_i
            else:
                df_final = pd.concat([df_final, df_i], ignore_index=True)
        # print(df_final.head(2))
        df_final['Value'] = df_final['Value'].astype(str)
        if not df_final is None:
            df_final.to_parquet(os.path.join(DATA_DIR, f'{split}/{fname}.parquet'))

In [ ]:
csv_to_parquet('train', TRAIN_FILES)
csv_to_parquet('test', TEST_FILES)

In [ ]:
def remove_files(keep_keywords = ['train', 'test', 'listfile']):
    for files, split in [(TRAIN_FILES, 'train'), (TEST_FILES, 'test')]:
        for filename in files:
            if not (filename[-3:] == 'csv') or filename == 'listfile.csv': 
                continue

            file_path = os.path.join(os.path.join(DATA_DIR, split), filename)
            
            if not any(keyword in filename.lower() for keyword in keep_keywords):
                os.remove(file_path)
                # print(f"Deleted: {filename}")

In [ ]:
remove_files()

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, 'train', 'listfile.csv'))

COMPUTE STATS

In [ ]:
def extract_stats(xs):
    xs = np.array(xs, dtype=float)
    x = np.arange(len(xs))
    slope, _, _, _, _ = linregress(x, xs)

    return pd.Series({
        'min': xs.min(),
        'max': xs.max(),
        'mean': xs.mean(),
        'std': xs.std(),
        'median': np.median(xs),
        'skew': skew(xs),
        'kurtosis': kurtosis(xs),
        'slope': slope,
        'qr_25': np.quantile(xs, 0.25),
        'qr_75': np.quantile(xs, 0.75),
    })

In [ ]:
EYE = ['Spontaneously', 'To Speech', 'To Pain'] 
MOTOR = ['No response', 'Localizes Pain', 'Flex-withdraws', 'Abnormal Flexion', 'Obeys Commands', 'Abnormal extension']
VERBAL = ['Incomprehensible sounds', 'No Response-ETT', 'Confused', 'Oriented', 'No Response', 'Inappropriate Words']

def parse_gcs(x):
    if x['Event'] == 'Glascow coma scale eye opening':
        return EYE.index(x['Value'])
    elif x['Event'] == 'Glascow coma scale motor response':
        return MOTOR.index(x['Value'])
    elif x['Event'] == 'Glascow coma scale verbal response':
        return VERBAL.index(x['Value'])
    else:
        return x['Value']

In [ ]:
def compute_stats(split, files):
    for file in files:
        # print(file[-7:])
        if not file[-7:] == 'parquet': 
            continue
        df = pd.read_parquet(os.path.join(DATA_DIR, split, file))
        if not 'Value' in df.columns: continue
        df['Value'] = df.apply(parse_gcs, axis=1)
        df['Value'] = pd.to_numeric(df['Value'])

        df = (df.sort_values('Hours')[['Event', 'Value']]).groupby('Event').agg(list).reset_index()
        df2 = df['Value'].apply(extract_stats).apply(pd.Series)

        df = df.drop(columns=['Value'])
        df = pd.concat([df, df2], axis=1)
        df.to_parquet(os.path.join(DATA_DIR, split, file))

In [ ]:
compute_stats('train', TRAIN_FILES)
compute_stats('test', TEST_FILES)

COMBINE

In [ ]:
def combine_files(split, files):
    df_combine = None
    for file in files:
        pat_id, episode, _ = file.split('_')
        visit_num = episode[7:]
        df = pd.read_parquet(os.path.join(DATA_DIR, split, file))
        df['patient_id'] = int(pat_id)
        df['visit_num'] = int(visit_num)

        if df_combine is None:
            df_combine = df
        else:
            df_combine = pd.concat([df, df_combine], ignore_index=True)
    return df_combine

In [ ]:
train_df = combine_files('train', TRAIN_FILES)
train_df.to_parquet(os.path.join(DATA_DIR, 'train.parquet'))

In [ ]:
test_df = combine_files('test', TEST_FILES)
test_df.to_parquet(os.path.join(DATA_DIR, 'test.parquet'))

In [ ]:
def remove_files(keep_keywords = ['train', 'test', 'listfile']):
    for files, split in [(TRAIN_FILES, 'train'), (TEST_FILES, 'test')]:
        for filename in files:
            file_path = os.path.join(os.path.join(DATA_DIR, split), filename)
            
            if not any(keyword in filename.lower() for keyword in keep_keywords):
                os.remove(file_path)
                # print(f"Deleted: {filename}")

In [ ]:
remove_files()

PIVOT

In [ ]:
def pivot_df(df):
    # df['visit_num'] = df.groupby(['patient_id', 'Event']).cumcount() + 1
    # df['pivot_col'] = df['Event'] + '_' + df['visit_num'].astype(str)
    # df = df[df['visit_num'] == 1]

    df_pivoted = df.pivot(index=['patient_id', 'visit_num'], columns='Event')
    df_pivoted.columns = [f'{col}_{event}' for col, event in df_pivoted.columns.swaplevel()]
    df_pivoted = df_pivoted.reindex(sorted(df_pivoted.columns), axis=1)
    df_pivoted = df_pivoted.reset_index()

    df_pivoted['patient_id'] = pd.to_numeric(df_pivoted['patient_id'])
    return df_pivoted

In [ ]:
train_df = pd.read_parquet(os.path.join(DATA_DIR, 'train.parquet'))
train_df = pivot_df(train_df)

In [ ]:
train_df.to_parquet(os.path.join(DATA_DIR, 'train.parquet'))

In [ ]:
test_df = pd.read_parquet(os.path.join(DATA_DIR, 'test.parquet'))
test_df = pivot_df(test_df)
test_df.to_parquet(os.path.join(DATA_DIR, 'test.parquet'))

DEMOGRAPHICS

In [ ]:
DATA_DIR = 'data/root'

In [ ]:
test_paths = os.listdir(os.path.join(DATA_DIR, 'test'))
train_paths = os.listdir(os.path.join(DATA_DIR, 'train'))

In [ ]:
RACES = []
GENDER = []
ETHNICITY = []

for path in test_paths:
    df1 = pd.read_csv(os.path.join(DATA_DIR, 'test', path, 'stays.csv'), usecols=['AGE', 'gender', 'race'])
    df2 = pd.read_csv(os.path.join(DATA_DIR, 'test', path, 'episode1.csv'), usecols=['Ethnicity'])
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    df1 = df1[~((df1['race']=='UNKNOWN') | (df1['race']=='OTHER') 
                | (df1['race']=='UNABLE TO OBTAIN') | (df1['race']=='PATIENT DECLINED TO ANSWER'))]
    RACES += list(df1['race'].unique())
    GENDER += list(df1['gender'].unique())
    ETHNICITY += list(df2['Ethnicity'].unique())
RACES = list(set(RACES))
GENDER = list(set(GENDER))
ETHNICITY = list(set(ETHNICITY))

RACES.sort()

In [ ]:
def get_demo(split, file_paths):
    demo_info = {}

    for path in file_paths:
        race, gender, age, ethnicity = None, None, None, None

        df1 = pd.read_csv(os.path.join(DATA_DIR, split, path, 'stays.csv'), usecols=['AGE', 'gender', 'race'])
        df2 = pd.read_csv(os.path.join(DATA_DIR, split, path, 'episode1.csv'), usecols=['Ethnicity'])
        df1 = df1.drop_duplicates()
        df2 = df2.drop_duplicates()
        
        df1 = df1[~((df1['race']=='UNKNOWN') | (df1['race']=='OTHER') 
                    | (df1['race']=='UNABLE TO OBTAIN') | (df1['race']=='PATIENT DECLINED TO ANSWER'))]
        
        if len(df1) >= 1:
            race = max(list(df1['race'].unique()), key=len)
            gender = max(list(df1['gender'].unique()), key=len)
            age = max(list(df1['AGE'].unique()))

        if len(df2) >= 1:
            ethnicity = max(list(df2['Ethnicity'].unique()))
        
        demo_info[path] = (race, ethnicity, gender, age)
    
    demo_df = pd.DataFrame(demo_info).T
    demo_df.columns = ['race', 'ethnicity', 'gender', 'age']

    demo_df['race_ind'] = demo_df['race'].apply(lambda x: RACES.index(x) if x in RACES else x)
    demo_df['gender_ind'] = demo_df['gender'].apply(lambda x: GENDER.index(x) if x in GENDER else x)

    demo_df['gender'] = demo_df['gender'].fillna('unknown')
    demo_df['race'] = demo_df['race'].fillna('unknown')
    demo_df['race_ind'] = demo_df['race_ind'].fillna(29)
    demo_df['gender_ind'] = demo_df['gender_ind'].fillna(2)

In [ ]:
train_paths = os.listdir(os.path.join(DATA_DIR, 'train'))
train_demo = get_demo('train', train_paths)
train_demo.to_parquet('data/root/train_demo.parquet')

In [ ]:
test_paths = os.listdir(os.path.join(DATA_DIR, 'test'))
test_demo = get_demo('test', test_paths)
test_demo.to_parquet('data/root/test_demo.parquet')

LABELS

In [ ]:
def process_label(label_df):
    label_df = label_df.rename(columns={'y_true':'Label'})
    label_df['patient_id'] = pd.to_numeric(label_df['stay'].apply(lambda x: x.split('_')[0]))
    label_df['visit_num'] = pd.to_numeric(label_df['stay'].apply(lambda x: x.split('_')[1][7:]))
    label_df = label_df[['patient_id', 'visit_num', 'Label']] # period_length
    return label_df
    

In [ ]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_labels.csv'))
train_df = process_label(train_df)
train_df.to_csv(os.path.join(DATA_DIR, 'train_labels.csv'), index=False)

In [ ]:
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test_labels.csv'))
test_df = process_label(test_df)
test_df.to_csv(os.path.join(DATA_DIR, 'test_labels.csv'), index=False)

GCS

In [ ]:
EYE, MOTOR, VERBAL = [], [], []

In [ ]:
def get_gcs_values(split, files):
    for file in files:
        df = pd.read_parquet(os.path.join(DATA_DIR, split, file))
        EYE += list(df[df['Event'] == 'Glascow coma scale eye opening']['Value'].unique())
        EYE = list(set(EYE))

        MOTOR += list(df[df['Event'] == 'Glascow coma scale motor response']['Value'].unique())
        MOTOR = list(set(MOTOR))

        VERBAL += list(df[df['Event'] == 'Glascow coma scale verbal response']['Value'].unique())
        VERBAL = list(set(VERBAL))

In [ ]:
train_files = os.listdir(os.path.join(DATA_DIR, 'train'))
get_gcs_values('train', train_files)

In [ ]:
test_files = os.listdir(os.path.join(DATA_DIR, 'test'))
get_gcs_values('test', test_files)